<a href="https://colab.research.google.com/github/jwoonge/ML-algorithms/blob/master/10_Text_Classification_using_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl https://colab.chainer.org/install | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0  12343      0 --:--:-- --:--:-- --:--:-- 12248
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.
+ pip install -q cupy-cuda100  chainer 
+ set +ex
Installation succeeded!


In [2]:
!pip install nltk
!pip install sklearn
!pip install pickle

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [3]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
nltk.download('wordnet')
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import cupy as cp
from google.colab import drive

drive.mount('/content/gdrive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
from zipfile import ZipFile
file_name = "movie_review.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [0]:
#'My_Drive/Colab_Notebooks/ML_assignments/10'
review_data = load_files(r"movie_review")
X, y = review_data.data, review_data.target

documents = []

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)
X_trian = cp.array(X_train) ; X_test = cp.array(X_test)
y_train = cp.array(y_train) ; y_test = cp.array(y_test)
y_train = cp.reshape(y_train,[len(y_train),1])
y_test = cp.reshape(y_test, [len(y_test),1])


In [6]:
print(np.shape(cp.asnumpy(X_train)))
print(np.shape(cp.asnumpy(X_test)))
print(np.shape(cp.asnumpy(y_train)))
print(np.shape(cp.asnumpy(y_test)))

(1401, 1500)
(601, 1500)
(1401, 1)
(601, 1)


In [0]:
def sigmoid(input):
    return 1/(1+cp.exp(-input))
def sigmoid_d(input):
    return sigmoid(input)*(1-sigmoid(input))

In [0]:
def loss(h, label, weights, lam):
    pre = cp.sum(cp.average(-label * cp.log(h) - (1-label)*cp.log(1-h), axis=0))
    n = 0
    theta_sum = 0

    for weight in weights:
        n += weight.size
        theta_sum += cp.sum(cp.square(weight))
    reg = lam/(2*n)*theta_sum

    return pre + reg

In [0]:
class classifier:
    def __init__(self, shape, lam, learning_rate):
        self.weights = []
        for i in range(len(shape)-1):
            self.weights.append(cp.random.randn(shape[i]+1, shape[i+1]))
        self.learning_rate = learning_rate
        self.lam = lam
        self.num_layer = len(self.weights)
        self.n = 0
        for weight in self.weights:
            self.n += weight.size
    
    def forward_pass(self, input, weights=[]):
        if weights==[]:
            weights = self.weights
        values = []
        values.append(cp.array(np.insert(cp.asnumpy(input), 0, 1, axis=1)))
        values.append(cp.dot(values[-1], weights[0]))
        values.append(cp.array(np.insert(cp.asnumpy(sigmoid(values[-1])),0,1,axis=1)))
        for i in range(1, len(weights)-1):
            values.append(cp.dot(values[-1], weights[i]))
            values.append(cp.array(np.insert(cp.asnumpy(sigmoid(values[-1])),0,1,axis=1)))
        values.append(cp.dot(values[-1], weights[-1]))
        values.append(sigmoid(values[-1]))
        
        return values

    def gradient_descent(self, forward_values, label):
        update_values = []
        m = len(label)
        mult = (sigmoid_d(forward_values[-2]) * ((1-label)/(1-forward_values[-1]) - label/forward_values[-1])).T
        layer = len(self.weights)-1
        update_values.append(cp.dot(mult, forward_values[2*layer]).T)
        
        for i in range(1, len(self.weights)):
            layer = len(self.weights) - i - 1
            mult = (cp.dot(mult.T, self.weights[layer+1][1:,:].T) * (sigmoid_d(forward_values[layer*2+1]))).T
            update_values.append(cp.dot(mult, forward_values[2*layer]).T)
        for i in range(len(self.weights)):
            layer = len(self.weights)-i-1
            self.weights[layer] = self.weights[layer] - self.learning_rate * (update_values[i]/m + self.lam*self.weights[layer]/self.n)

    def optimize(self, data_train, label_train, data_test, label_test, epoch):
        forward_train = self.forward_pass(data_train)
        forward_test = self.forward_pass(data_test)
        h_train = forward_train[-1]
        h_test = forward_test[-1]
        y_pred_train = np.where(cp.asnumpy(h_train)<0.5, 0, 1)
        y_pred_test = np.where(cp.asnumpy(h_test)<0.5, 0, 1)
        label_train_np = cp.asnumpy(label_train)
        label_test_np = cp.asnumpy(label_test)
        
        self.loss_train = [loss(h_train, label_train, self.weights, self.lam)]
        self.loss_test = [loss(h_test, label_test, self.weights, self.lam)]
        self.accuracy_train = [accuracy_score(label_train_np, y_pred_train)]
        self.accuracy_test = [accuracy_score(label_test_np, y_pred_test)]
        self.report_train = [classification_report(label_train_np, y_pred_train)]
        self.report_test = [classification_report(label_test_np, y_pred_test)]
        self.confusion_matrix_train = [confusion_matrix(label_train_np, y_pred_train)]
        self.confusion_matrix_test = [confusion_matrix(label_test_np, y_pred_test)]
        
        for i in range(epoch-1):

            self.gradient_descent(forward_train, label_train)
            forward_train = self.forward_pass(data_train)
            forward_test = self.forward_pass(data_test)
            h_train = forward_train[-1]
            h_test = forward_test[-1]
            y_pred_train = np.where(cp.asnumpy(h_train)<0.5, 0, 1)
            y_pred_test = np.where(cp.asnumpy(h_test)<0.5, 0, 1)

            self.loss_train.append(loss(h_train, label_train, self.weights, self.lam))
            self.loss_test.append(loss(h_test, label_test, self.weights, self.lam))
            self.accuracy_train.append(accuracy_score(label_train_np, y_pred_train))
            self.accuracy_test.append(accuracy_score(label_test_np, y_pred_test))
            self.report_train.append(classification_report(label_train_np, y_pred_train))
            self.report_test.append(classification_report(label_test_np, y_pred_test))
            self.confusion_matrix_train.append(confusion_matrix(label_train_np, y_pred_train))
            self.confusion_matrix_test.append(confusion_matrix(label_test_np, y_pred_test))


            print(i," LOS_T:", format(float(self.loss_train[-1]),'.5f'), "\tACC_T:", format(float(self.accuracy_train[-1]),'.5f'), "| LOS_V:", format(float(self.loss_test[-1]),'.5f'), "ACC_V:" ,format(float(self.accuracy_test[-1]),'.5f'), format(self.lam/self.n,'.10f'), self.n)

In [31]:
Classifier = classifier([1500, 56, 24, 1], lam=10, learning_rate=1)
Classifier.optimize(X_train, y_train, X_test, y_test, 2000)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0  LOS_T: 6.94779 	ACC_T: 0.49893 | LOS_V: 6.94851 ACC_V: 0.50250 0.0001170289 85449
1  LOS_T: 6.91185 	ACC_T: 0.50107 | LOS_V: 6.92750 ACC_V: 0.49750 0.0001170289 85449
2  LOS_T: 6.76676 	ACC_T: 0.49893 | LOS_V: 6.76661 ACC_V: 0.50250 0.0001170289 85449
3  LOS_T: 6.66729 	ACC_T: 0.50107 | LOS_V: 6.68167 ACC_V: 0.49750 0.0001170289 85449
4  LOS_T: 6.58841 	ACC_T: 0.49893 | LOS_V: 6.58688 ACC_V: 0.50250 0.0001170289 85449
5  LOS_T: 6.50995 	ACC_T: 0.50107 | LOS_V: 6.52427 ACC_V: 0.49750 0.0001170289 85449
6  LOS_T: 6.43605 	ACC_T: 0.49893 | LOS_V: 6.43291 ACC_V: 0.50250 0.0001170289 85449
7  LOS_T: 6.39409 	ACC_T: 0.50107 | LOS_V: 6.40820 ACC_V: 0.49750 0.0001170289 85449
8  LOS_T: 6.32753 	ACC_T: 0.49893 | LOS_V: 6.32301 ACC_V: 0.50250 0.0001170289 85449
9  LOS_T: 6.30205 	ACC_T: 0.50107 | LOS_V: 6.31607 ACC_V: 0.49750 0.0001170289 85449
10  LOS_T: 6.25053 	ACC_T: 0.49893 | LOS_V: 6.24513 ACC_V: 0.50250 0.0001170289 85449
11  LOS_T: 6.22889 	ACC_T: 0.50107 | LOS_V: 6.24308 ACC_V: 0.497

In [0]:
print(np.shape(X_train))
print(len(y_train))
y_train = cp.reshape(y_train, (len(y_train),1))
print(np.shape(y_train))
#correct = h[np.abs(우리꺼 - 정답)<0.5]